In [67]:
import pandas as pd
from py2neo import Graph
import matplotlib
from sklearn.preprocessing import StandardScaler


AttributeError: module 'scipy' has no attribute '_lib'

In [4]:
graph = Graph(host="localhost", auth=("neo4j", "test123"))

In [39]:
query = "MATCH (clan:Clan)--(standing:War_Standing)-[r]-(war:War) MATCH (participant:War_Participant)--(standing) RETURN war.war_end_time as Time,clan.name as Clan, sum(participant.cards_earned) as cards, standing.participants as Participants, standing.wins as Wins, standing.battles_played as Played, r.standing as Result"

war_results = graph.run(query).to_data_frame()
war_results.shape

(472, 7)

In [ ]:
matplotlib.pyplot.scatter(war_results[])